In [103]:
import pandas as pd
import os
import numpy as np
import matplotlib.ticker as mtick
import arcpy

# Import the required ArcGIS API for Python modules
import arcgis
gis = arcgis.GIS()

from IPython.display import display, Markdown

In [104]:
#scenario name
#allow change of scen_name
#scenario names must have an associated class parameters file in the config folder
scen_names = ["_55","_70","_85","_orig"]

#get root folder
dir_working = os.getcwd()
print(dir_working)

#define folders
dir_data     = os.path.join(dir_working, "data"        )
dir_config   = os.path.join(dir_working, "config"      )
dir_process  = os.path.join(dir_working, "intermediate")
dir_results  = os.path.join(dir_working, "results"     )
dir_inputs   = os.path.join(dir_working, "input"       )
dir_webapp   = os.path.join(dir_working, "forWebApp"   )

ProcessGDBname = "process.gdb"
FinalGDBname   = "parcel_redevelopment_potential.gdb"

ProcessGDBpath = os.path.join(dir_process, ProcessGDBname)
FinalGDBpath   = os.path.join(dir_results, FinalGDBname  )

ParcelsDevCategory = os.path.join(dir_process, ProcessGDB + "\ParcelsAA_DevCategory" + scen_name)
ParcelsWebApp      = os.path.join(dir_webapp , WebAppGDB  + "\AAParcels"             + scen_name)
DevCatWebApp       = os.path.join(dir_webapp , WebAppGDB  + "\AADevCategories"       + scen_name)

#name of new data features
AnalysisAreas      = os.path.join(dir_process, "analysisareas.shp"        )
AnalysisAreas_IDs  = os.path.join(dir_process, "analysisareas_ids.shp"    )
ParcelsAA          = os.path.join(dir_process, "parcelsaa.shp"            )
AnalysisAreasTable = os.path.join(dir_results, "areas.csv"                )

def deleteIfExists(obj):
    if arcpy.Exists(obj): arcpy.Delete_management(obj)
        
                
#if processing geodatabase doesn't exist, create it
print("Checking if " + FinalGDBname + " exists...")
if not arcpy.Exists(FinalGDBpath):
    print("Creating " + FinalGDBname + "...")
    arcpy.management.CreateFileGDB(dir_results, FinalGDBname)
else:
    print(FinalGDBname + " exists...")

print("done")

E:\GitHub\Parcel-Redevelopment-Potential
Checking if parcel_redevelopment_potential.gdb exists...
parcel_redevelopment_potential.gdb exists...
done


In [105]:
AnalysisAreas

'E:\\GitHub\\Parcel-Redevelopment-Potential\\intermediate\\analysisareas.shp'

In [106]:
sdf_AA = pd.DataFrame.spatial.from_featureclass(AnalysisAreas)
sdf_AA

,FID,ClassName,code,ClassCode,Shape_Leng,Shape_Area,AnalysisAr,Analysis_1,SHAPE
0,0,CRT|NA,,CRT|NA,504.262253,3740.082039,CRT|NA,CRT|NA,"{""rings"": [[[444099.3367999997, 4452551.347999..."
1,1,CRT|NA,,CRT|NA,5563.892221,592601.042333,CRT|NA,CRT|NA,"{""rings"": [[[438376.81769999955, 4459637.032],..."
2,2,CRT|NA,,CRT|NA,2570.761620,278930.897498,CRT|NA,CRT|NA,"{""rings"": [[[430849.71740000043, 4469533.5944]..."
3,3,CRT|NA,,CRT|NA,2134.667772,259008.671655,CRT|NA,CRT|NA,"{""rings"": [[[423912.7335000001, 4474824.7914],..."
4,4,CRT|NA,,CRT|NA,1361.032628,21751.918752,CRT|NA,CRT|NA,"{""rings"": [[[423670.5016999999, 4485868.4681],..."
...,...,...,...,...,...,...,...,...,...
13204,13204,,h1,h1,531.497049,131.721210,h1,h1,"{""rings"": [[[429644.14269999973, 4481668.1929]..."
13205,13205,LRT|NA,d4,LRT|NA,482.530003,0.208915,LRT|NA,LRT|NA,"{""rings"": [[[415404.0895999996, 4492683.829600..."
13206,13206,LRT|NA,h1,LRT|NA,482.530003,0.208915,LRT|NA,LRT|NA,"{""rings"": [[[415404.0895999996, 4492683.829600..."
13207,13207,NONTOD|City Center,b3,NONTOD|City Center,238.410394,0.251195,NONTOD|City Center,NONTOD|City Center,"{""rings"": [[[410819.7604, 4557518.294199999], ..."


# Parcels

In [17]:
sdf_ParcelsAA = pd.DataFrame.spatial.from_featureclass(ParcelsAA)
sdf_ParcelsAA.groupby(['area_id','class_id'], as_index=False).agg({"parcel_id": [np.size], "job_spaces": [np.sum], "resunits":[np.sum], "acres":[np.sum]})

,area_id,class_id,parcel_id,job_spaces,resunits,acres
,,,size,sum,sum,sum
0,0,a1,13095.0,226.688223,1392.344480,18537.668449
1,1,a2,12941.0,451.564918,3169.517601,15433.671502
2,2,a3,16402.0,1396.846492,5504.083008,19063.441789
3,3,a4,11839.0,977.832660,3610.321731,8915.357612
4,4,a5,5311.0,974.335819,2813.550268,5889.372093
5,5,b1,8835.0,1077.906305,2917.467340,10481.991379
6,6,b2,21306.0,828.742176,12267.218797,14656.221448
7,7,b3,23158.0,2457.831194,11072.366567,8799.637586
8,8,b4,18373.0,1627.819803,10161.046634,9472.022185


In [18]:
sdf_ParcelsAA['county_id']

0          4.0
1          4.0
2          4.0
3          4.0
4          4.0
          ... 
1027903    3.0
1027904    3.0
1027905    3.0
1027906    3.0
1027907    3.0
Name: county_id, Length: 1027908, dtype: float64

# Land Use

In [19]:
display(sdf_ParcelsAA.columns)
sdf_ParcelsAA = sdf_ParcelsAA.rename(columns={"lu_type": "lu_type_ex"})
display(sdf_ParcelsAA.columns)

Index(['FID', 'Shape_Leng', 'Shape_Area', 'fid_analys', 'area_id', 'class_id',
       'fid_parcel', 'parcel_id', 'parcel_id_', 'basebldg', 'parcel_sqf',
       'parcel_acr', 'county_id', 'max_dua', 'max_far', 'max_height',
       'lu_type', 'bldgtype', 'yearbuilt', 'resunits', 'job_spaces', 'sf_res',
       'sf_com', 'sfvalue_re', 'sfvalue_co', 'value_res', 'value_com',
       'parcel_a_1', 'piece_acre', 'piece_port', 'acres', 'SHAPE'],
      dtype='object')

Index(['FID', 'Shape_Leng', 'Shape_Area', 'fid_analys', 'area_id', 'class_id',
       'fid_parcel', 'parcel_id', 'parcel_id_', 'basebldg', 'parcel_sqf',
       'parcel_acr', 'county_id', 'max_dua', 'max_far', 'max_height',
       'lu_type_ex', 'bldgtype', 'yearbuilt', 'resunits', 'job_spaces',
       'sf_res', 'sf_com', 'sfvalue_re', 'sfvalue_co', 'value_res',
       'value_com', 'parcel_a_1', 'piece_acre', 'piece_port', 'acres',
       'SHAPE'],
      dtype='object')

In [20]:
sdf_ParcelsAA.loc[(sdf_ParcelsAA['lu_type_ex'] != "Undevelopable") & (sdf_ParcelsAA['bldgtype'] == 'Residential'), 'lu_type_ex'] = "Residential"
sdf_ParcelsAA.loc[(sdf_ParcelsAA['lu_type_ex'] != "Undevelopable") & (sdf_ParcelsAA['bldgtype'] == 'Commercial' ), 'lu_type_ex'] = "Commercial"
sdf_ParcelsAA.loc[(sdf_ParcelsAA['lu_type_ex'] != "Undevelopable") & ((sdf_ParcelsAA['bldgtype'] != 'Residential') & (sdf_ParcelsAA['bldgtype'] != 'Commercial')), 'lu_type_ex'] = "Developable"

# Building Age

In [21]:
sdf_ParcelsAA.columns

Index(['FID', 'Shape_Leng', 'Shape_Area', 'fid_analys', 'area_id', 'class_id',
       'fid_parcel', 'parcel_id', 'parcel_id_', 'basebldg', 'parcel_sqf',
       'parcel_acr', 'county_id', 'max_dua', 'max_far', 'max_height',
       'lu_type_ex', 'bldgtype', 'yearbuilt', 'resunits', 'job_spaces',
       'sf_res', 'sf_com', 'sfvalue_re', 'sfvalue_co', 'value_res',
       'value_com', 'parcel_a_1', 'piece_acre', 'piece_port', 'acres',
       'SHAPE'],
      dtype='object')

In [22]:
#set building age with base year 2019
sdf_ParcelsAA['BldAge'] = 0
sdf_ParcelsAA.loc[((sdf_ParcelsAA['yearbuilt'] > 0)), 'bld_age'] = 2019 - sdf_ParcelsAA['yearbuilt']
display(sdf_ParcelsAA)
display(sdf_ParcelsAA.columns)

,FID,Shape_Leng,Shape_Area,fid_analys,area_id,class_id,fid_parcel,parcel_id,parcel_id_,basebldg,...,sfvalue_co,value_res,value_com,parcel_a_1,piece_acre,piece_port,acres,SHAPE,BldAge,bld_age
0,0,172.932395,1681.967937,1,17,CRT|NA,19736,645747.0,645747.0,1.0,...,0.0,0.0,0.0,3.220934,0.415623,0.129038,0.415623,"{""rings"": [[[444099.3367999997, 4452551.347999...",0,39.0
1,1,30.103240,38.790622,1,17,CRT|NA,24447,598653.0,598653.0,1.0,...,0.0,0.0,0.0,5.987770,0.009585,0.001601,0.009585,"{""rings"": [[[444148.9570000004, 4452591.1204],...",0,39.0
2,2,167.068897,677.810606,1,17,CRT|NA,37976,598283.0,598283.0,0.0,...,0.0,0.0,0.0,0.557976,0.167491,0.300175,0.167491,"{""rings"": [[[443767.41139999963, 4452576.75919...",0,NaN
3,3,32.992324,53.164504,1,17,CRT|NA,131771,642522.0,642522.0,0.0,...,0.0,0.0,0.0,0.124928,0.013137,0.105158,0.013137,"{""rings"": [[[444148.93620000035, 4452591.10160...",0,NaN
4,4,115.593224,468.612707,1,17,CRT|NA,155153,646774.0,646774.0,1.0,...,0.0,0.0,0.0,2.883256,0.115797,0.040162,0.115797,"{""rings"": [[[443768.3559999997, 4452501.2521],...",0,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027903,1027903,77.565301,218.370145,11804,37,h1,648134,549671.0,549671.0,0.0,...,0.0,0.0,0.0,0.171895,0.053960,0.313916,0.053960,"{""rings"": [[[416285.11959999986, 4572384.52040...",0,NaN
1027904,1027904,77.565301,218.370145,11804,37,h1,665078,546641.0,546641.0,0.0,...,0.0,0.0,0.0,0.171895,0.053960,0.313916,0.053960,"{""rings"": [[[416285.11959999986, 4572384.52040...",0,NaN
1027905,1027905,77.565301,218.370145,11804,37,h1,665079,545690.0,545690.0,0.0,...,0.0,0.0,0.0,0.171895,0.053960,0.313916,0.053960,"{""rings"": [[[416285.11959999986, 4572384.52040...",0,NaN
1027906,1027906,77.565301,218.370145,11804,37,h1,685145,548151.0,548151.0,0.0,...,0.0,0.0,0.0,0.171895,0.053960,0.313916,0.053960,"{""rings"": [[[416285.11959999986, 4572384.52040...",0,NaN


Index(['FID', 'Shape_Leng', 'Shape_Area', 'fid_analys', 'area_id', 'class_id',
       'fid_parcel', 'parcel_id', 'parcel_id_', 'basebldg', 'parcel_sqf',
       'parcel_acr', 'county_id', 'max_dua', 'max_far', 'max_height',
       'lu_type_ex', 'bldgtype', 'yearbuilt', 'resunits', 'job_spaces',
       'sf_res', 'sf_com', 'sfvalue_re', 'sfvalue_co', 'value_res',
       'value_com', 'parcel_a_1', 'piece_acre', 'piece_port', 'acres', 'SHAPE',
       'BldAge', 'bld_age'],
      dtype='object')

# Parcel Value

In [23]:
sdf_ParcelsAA.columns

Index(['FID', 'Shape_Leng', 'Shape_Area', 'fid_analys', 'area_id', 'class_id',
       'fid_parcel', 'parcel_id', 'parcel_id_', 'basebldg', 'parcel_sqf',
       'parcel_acr', 'county_id', 'max_dua', 'max_far', 'max_height',
       'lu_type_ex', 'bldgtype', 'yearbuilt', 'resunits', 'job_spaces',
       'sf_res', 'sf_com', 'sfvalue_re', 'sfvalue_co', 'value_res',
       'value_com', 'parcel_a_1', 'piece_acre', 'piece_port', 'acres', 'SHAPE',
       'BldAge', 'bld_age'],
      dtype='object')

In [24]:
#set building age with base year 2019
sdf_ParcelsAA['ValuePerAcre'] = 0
sdf_ParcelsAA.loc[((sdf_ParcelsAA['lu_type_ex'] == 'Residential')), 'ValuePerAcre'] = sdf_ParcelsAA['value_res'] / sdf_ParcelsAA['acres']
sdf_ParcelsAA.loc[((sdf_ParcelsAA['lu_type_ex'] == 'Commercial' )), 'ValuePerAcre'] = sdf_ParcelsAA['value_com'] / sdf_ParcelsAA['acres']

#classify existing acreage by existing land use

sdf_ParcelsAA['acres_res'] = 0.0
sdf_ParcelsAA['acres_com'] = 0.0

sdf_ParcelsAA.loc[((sdf_ParcelsAA['lu_type_ex'] == 'Residential')), 'acres_res'] = sdf_ParcelsAA['acres']
sdf_ParcelsAA.loc[((sdf_ParcelsAA['lu_type_ex'] == 'Commercial' )), 'acres_com'] = sdf_ParcelsAA['acres']

display(sdf_ParcelsAA)

,FID,Shape_Leng,Shape_Area,fid_analys,area_id,class_id,fid_parcel,parcel_id,parcel_id_,basebldg,...,parcel_a_1,piece_acre,piece_port,acres,SHAPE,BldAge,bld_age,ValuePerAcre,acres_res,acres_com
0,0,172.932395,1681.967937,1,17,CRT|NA,19736,645747.0,645747.0,1.0,...,3.220934,0.415623,0.129038,0.415623,"{""rings"": [[[444099.3367999997, 4452551.347999...",0,39.0,0.0,0.0,0.415623
1,1,30.103240,38.790622,1,17,CRT|NA,24447,598653.0,598653.0,1.0,...,5.987770,0.009585,0.001601,0.009585,"{""rings"": [[[444148.9570000004, 4452591.1204],...",0,39.0,0.0,0.0,0.009585
2,2,167.068897,677.810606,1,17,CRT|NA,37976,598283.0,598283.0,0.0,...,0.557976,0.167491,0.300175,0.167491,"{""rings"": [[[443767.41139999963, 4452576.75919...",0,NaN,0.0,0.0,0.000000
3,3,32.992324,53.164504,1,17,CRT|NA,131771,642522.0,642522.0,0.0,...,0.124928,0.013137,0.105158,0.013137,"{""rings"": [[[444148.93620000035, 4452591.10160...",0,NaN,0.0,0.0,0.000000
4,4,115.593224,468.612707,1,17,CRT|NA,155153,646774.0,646774.0,1.0,...,2.883256,0.115797,0.040162,0.115797,"{""rings"": [[[443768.3559999997, 4452501.2521],...",0,39.0,0.0,0.0,0.115797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027903,1027903,77.565301,218.370145,11804,37,h1,648134,549671.0,549671.0,0.0,...,0.171895,0.053960,0.313916,0.053960,"{""rings"": [[[416285.11959999986, 4572384.52040...",0,NaN,0.0,0.0,0.000000
1027904,1027904,77.565301,218.370145,11804,37,h1,665078,546641.0,546641.0,0.0,...,0.171895,0.053960,0.313916,0.053960,"{""rings"": [[[416285.11959999986, 4572384.52040...",0,NaN,0.0,0.0,0.000000
1027905,1027905,77.565301,218.370145,11804,37,h1,665079,545690.0,545690.0,0.0,...,0.171895,0.053960,0.313916,0.053960,"{""rings"": [[[416285.11959999986, 4572384.52040...",0,NaN,0.0,0.0,0.000000
1027906,1027906,77.565301,218.370145,11804,37,h1,685145,548151.0,548151.0,0.0,...,0.171895,0.053960,0.313916,0.053960,"{""rings"": [[[416285.11959999986, 4572384.52040...",0,NaN,0.0,0.0,0.000000


# Development Categories

In [108]:
scen_names

for scen_name in scen_names:
    
    display ('Processing ' + scen_name + ". Calculating parcel eligibility.")
    ClassParametersScenario = os.path.join(dir_config, r"class_parameters" + scen_name + ".csv")

    df_ClassParam = pd.read_csv(ClassParametersScenario)
    #display(df_ClassParam)
    
    
    #ReDevelop, NewDevelop, NoDevelop

    sdf_ParcelsAAwParam = pd.DataFrame.merge(sdf_ParcelsAA,df_ClassParam,left_on='class_id',right_on="ClassID",how='left')
    #display(sdf_ParcelsAAwParam)

    #undeveloped
    sdf_ParcelsDevCat = sdf_ParcelsAAwParam.copy()
    sdf_ParcelsDevCat['DevCategory'] = 'Remain'
    sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['lu_type_ex'] == 'Developable'), 'DevCategory'] = "Developable"

    #initialize column
    sdf_ParcelsDevCat['DevCategoryAge'] = ''

    #check if redevelopable - step 1: building age
    #residential
    sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['bld_age'] >= sdf_ParcelsDevCat['RedevBldgAgeRes_Low']) & (sdf_ParcelsDevCat['BldAge'] <= sdf_ParcelsDevCat['RedevBldgAgeRes_High']) & (sdf_ParcelsDevCat['lu_type_ex'] == 'Residential'), 'DevCategoryAge'] = "Redevelopable"
    #non-residential
    sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['bld_age'] >= sdf_ParcelsDevCat['RedevBldgAgeCom_Low']) & (sdf_ParcelsDevCat['BldAge'] <= sdf_ParcelsDevCat['RedevBldgAgeCom_High']) & (sdf_ParcelsDevCat['lu_type_ex'] != 'Residential'), 'DevCategoryAge'] = "Redevelopable"

    #initialize column
    sdf_ParcelsDevCat['DevCategoryValue'] = ''

    #check if redevelopable - step 2: value
    #residential
    sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['ValuePerAcre'] > 0) & (sdf_ParcelsDevCat['ValuePerAcre'] <= sdf_ParcelsDevCat['RedevValuePerAcreRes']) & (sdf_ParcelsDevCat['lu_type_ex'] == 'Residential'), 'DevCategoryValue'] = "Redevelopable"
    #commercial
    sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['ValuePerAcre'] > 0) & (sdf_ParcelsDevCat['ValuePerAcre'] <= sdf_ParcelsDevCat['RedevValuePerAcreCom']) & (sdf_ParcelsDevCat['lu_type_ex'] == 'Commercial' ), 'DevCategoryValue'] = "Redevelopable"

    #depending on AND or OR defenition, determine whether a parcel is eligible for redevelopment
    sdf_ParcelsDevCat.loc[ (sdf_ParcelsDevCat['RedevAndOr'] =='AND') &  (sdf_ParcelsDevCat['DevCategoryAge'] == 'Redevelopable') & (sdf_ParcelsDevCat['DevCategoryValue'] == 'Redevelopable')  , 'DevCategory'] = "Redevelopable"
    sdf_ParcelsDevCat.loc[((sdf_ParcelsDevCat['RedevAndOr'] =='OR' ) & ((sdf_ParcelsDevCat['DevCategoryAge'] == 'Redevelopable') | (sdf_ParcelsDevCat['DevCategoryValue'] == 'Redevelopable'))), 'DevCategory'] = "Redevelopable"

    #undevelopable parcels
    sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['lu_type_ex'] == 'Undevelopable'), 'DevCategory'] = "Undevelopable"


    #single-family redevelopment additional criteria (redeveloped parcel must include enough area for at least 1.0 more unit than original)

    #calculate redev potential for SF units... resunits range from 0.95 to 1.05 to account for rounding error
    sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['DevCategory'] == 'Redevelopable') & (sdf_ParcelsDevCat['SFRedevFullAdd'] =='Yes') & (sdf_ParcelsDevCat['resunits'] > 0.95) & (sdf_ParcelsDevCat['resunits'] < 1.05), 'SFRedevAdd'] = sdf_ParcelsDevCat['CapacityRes_DUA'] * (sdf_ParcelsDevCat['acres'] * (1-sdf_ParcelsDevCat['PercentOpenSpace'])) - sdf_ParcelsDevCat['resunits']

    #change to 'Remain' for SF parcels with not enough redev space for additional units
    sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['DevCategory'] == 'Redevelopable') & (sdf_ParcelsDevCat['SFRedevFullAdd'] =='Yes') & (sdf_ParcelsDevCat['resunits'] > 0.95) & (sdf_ParcelsDevCat['resunits'] < 1.05) & (sdf_ParcelsDevCat['SFRedevAdd'] < 1.0), 'DevCategory']      = 'Remain'

    #changed to Remain
    sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['SFRedevAdd'] < 1.0), 'DevCategory_Flag'] = 'SFRedevAdd_NoPotential'

    #create layer names for intermediate feature classes
    ParcelsDevCategory          = os.path.join(ProcessGDBpath, "ParAA_DevCat" + scen_name)
    ParcelsDevCategory_simple   = os.path.join(ProcessGDBpath, "ParAA_DevCat" + scen_name + "_simple")
    
    #create layer names for final feature class
    ParcelsDevCategory_final    = os.path.join(FinalGDBpath,"Parcel_Redevelopment_Potential" + scen_name)
    
    deleteIfExists(ParcelsDevCategory)
    deleteIfExists(ParcelsDevCategory_simple)
    deleteIfExists(ParcelsDevCategory_final)

    #limit to only needed fields
    sdf_ParcelsDevCat = sdf_ParcelsDevCat[['parcel_id','DevCategory', 'DevCategoryAge',
           'DevCategoryValue', 'SFRedevAdd', 'DevCategory_Flag','SHAPE']]

    #rename columns -- sometimes crazy if column names are "sanitized"
    sdf_ParcelsDevCat.columns = ('parcel_id','dev_cat', 'dev_cat_age',
           'dev_cat_value', 'sf_redev_add', 'dev_cat_flag','SHAPE')
    
    #export to GeoDB
    display("exporting " + ParcelsDevCategory)

    #export first layer
    sdf_ParcelsDevCat.spatial.to_featureclass(location=ParcelsDevCategory)
    
    display("simplyfing " + scen_name)
    
    #simplify layer... removes some polygon vertices... parcels layer is so large, this reduces file size quite a bit
    #https://pro.arcgis.com/en/pro-app/latest/tool-reference/cartography/simplify-polygon.htm
    arcpy.cartography.SimplifyPolygon(ParcelsDevCategory, ParcelsDevCategory_simple, 'POINT_REMOVE', 1)

    display("dissolving " + scen_name)
    #dissovle, but creating dissolved layer, the .dbf portion of shapefile is super small
    arcpy.management.Dissolve(ParcelsDevCategory_simple, ParcelsDevCategory_final, ["dev_cat"])

    display(ParcelsDevCategory_final + " is done")
    
display('All Done!')

'Processing _55'

'exporting E:\\GitHub\\Parcel-Redevelopment-Potential\\intermediate\\process.gdb\\ParAA_DevCat_55'

'simplyfing _55'

'dissolving _55'

'Processing _70'

...

'dissolving _85'

'Processing _orig'

'exporting E:\\GitHub\\Parcel-Redevelopment-Potential\\intermediate\\process.gdb\\ParAA_DevCat_orig'

'simplyfing _orig'

'dissolving _orig'

In [ ]:
sdf_ParcelsDevCat

In [ ]:
display(df_ClassParam)
df_ClassParam.dtypes

In [ ]:
#aggregate parcels into analysis areas / classes
df_AADevCat = sdf_ParcelsDevCat.groupby(['area_id','class_id','county_id','dev_category'],as_index=False).agg({"parcel_id": [np.size], "job_spaces": [np.sum], "resunits":[np.sum], "acres":[np.sum], "sf_res":[np.sum], "sf_com":[np.sum], "acres_res":[np.sum], "acres_com":[np.sum]})
df_AADevCat.columns = df_AADevCat.columns.droplevel(1)
display(df_AADevCat)


df_AADevCat.to_csv(os.path.join(dir_results, r'AnalysisAreas_byDevCategory' + scen_name + '.csv'),index=False)
